# Câu 6: Association Rules – BAKERY (1.0 điểm)
Use dataset **75000** (select one file in this folder that is suitable for you) to build the model to identify sets of items that are frequently bought together (please use Flavor and Food name (in **goods.csv**) instead of Id).
<br> Read more information here:
<br> Dataset: http://users.csc.calpoly.edu/~dekhtyar/466-Spring2018/labs/lab01.html

In [1]:
#@title Install Pakages
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [3]:
#@title Mount to drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd '/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Du lieu cung cap/75000'

/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Du lieu cung cap/75000


In [5]:
import pyspark

In [54]:
#@title Import pakages
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import collect_list, col, count, collect_set
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf, isnan
from datetime import datetime
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode, when
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col, concat


In [10]:
sc = SparkContext()
spark = SparkSession.builder.appName('Cau3').getOrCreate()

In [11]:
schema = StructType() \
      .add("TransactionID",IntegerType(),True) \
      .add("FoodID",IntegerType(),True) \
      .add("FlavorID",IntegerType(),True) 
      
df = spark.read.csv('75000i.csv', schema = schema, header = True, inferSchema=True)
df.show(5)

In [13]:
df.count()

266208

In [14]:
# Preprocessing data

In [17]:
df.printSchema()

root
 |-- TransactionID: integer (nullable = true)
 |-- FoodID: integer (nullable = true)
 |-- FlavorID: integer (nullable = true)



In [26]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).toPandas().T


,0
TransactionID,0
FoodID,0
FlavorID,0


In [ ]:
# Nhận xét: Dữ liệu không có nan

In [22]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T

,0
TransactionID,0
FoodID,0
FlavorID,0


In [ ]:
# Nhận xét giá trị không có giá trị null

## Pre-processing data

In [28]:
df.createOrReplaceTempView("order_items_train")

In [29]:
products = spark.sql("select distinct FlavorID from order_items_train")
products.count()

50

In [30]:
# Nhận xét: Có 50 item đúng với file goods.csv trong bài toán

In [31]:
rawData = spark.sql("select * from order_items_train")
baskets = rawData.groupBy('TransactionID').agg(collect_set('FlavorID').alias('items'))
baskets.createOrReplaceTempView('baskets')

In [32]:
baskets.show(5, truncate=False)

+-------------+----------------------+
|TransactionID|items                 |
+-------------+----------------------+
|148          |[33, 27, 9, 46, 28, 4]|
|463          |[17, 14]              |
|471          |[9, 37, 34, 20]       |
|496          |[15, 6, 47, 26]       |
|833          |[12, 5, 21]           |
+-------------+----------------------+
only showing top 5 rows



In [33]:
# Nhận xét đã chuyển bài toán qua dạng hợp lý số giao dịch và sản phẩm mua

In [35]:
# Khởi tạo thuật toán
from pyspark.ml.fpm import FPGrowth

In [36]:
fpGrowth = FPGrowth(itemsCol='items', minSupport=0.003, minConfidence= 0.003)
model = fpGrowth.fit(baskets)

In [37]:
# Display frequent itemsets
model.freqItemsets.show()

+--------+----+
|   items|freq|
+--------+----+
|     [7]|8193|
|    [45]|7700|
| [45, 7]|2367|
|    [28]|7556|
|[28, 45]| 387|
| [28, 7]| 383|
|    [18]|6987|
|[18, 28]| 393|
|[18, 45]| 318|
| [18, 7]| 321|
|     [4]|6948|
| [4, 28]| 465|
| [4, 45]| 372|
| [4, 18]| 402|
|  [4, 7]| 378|
|    [35]|6943|
|[35, 28]| 381|
| [35, 4]| 388|
|[35, 45]| 309|
|[35, 18]|3982|
+--------+----+
only showing top 20 rows



In [38]:
# transform
mostPopularItemInABasket = model.transform(baskets)

In [39]:
mostPopularItemInABasket.show()

+-------------+--------------------+--------------------+
|TransactionID|               items|          prediction|
+-------------+--------------------+--------------------+
|          148|[33, 27, 9, 46, 2...|[35, 22, 45, 18, ...|
|          463|            [17, 14]|[27, 28, 35, 4, 2...|
|          471|     [9, 37, 34, 20]|[19, 33, 27, 17, ...|
|          496|     [15, 6, 47, 26]|[27, 33, 1, 28, 3...|
|          833|         [12, 5, 21]|[19, 33, 27, 17, ...|
|         1088| [27, 35, 3, 18, 40]|[28, 4, 22, 45, 4...|
|         1238|        [19, 32, 18]|[28, 45, 7, 4, 35...|
|         1342|         [49, 17, 8]|[19, 27, 33, 1, 2...|
|         1580|        [12, 31, 36]|[48, 19, 33, 27, ...|
|         1591|             [1, 19]|[27, 33, 28, 37, ...|
|         1645|         [15, 49, 7]|[27, 33, 1, 28, 3...|
|         1829|[15, 49, 38, 6, 7...|[27, 33, 1, 28, 3...|
|         1959|[9, 1, 18, 4, 22,...|[28, 45, 7, 35, 4...|
|         2122|             [5, 22]|[27, 1, 28, 35, 1...|
|         2142

## Sử dụng tên sản phẩm thay cho ID sản phẩm

In [40]:
goods_df = spark.read.csv("goods.csv", header=True, inferSchema=True)

In [41]:
goods_df.show(5, truncate=False)

+---+------------+------+-----+------+
|Id |Flavor      |Food  |Price|Type  |
+---+------------+------+-----+------+
|0  |'Chocolate' |'Cake'|8.95 |'Food'|
|1  |'Lemon'     |'Cake'|8.95 |'Food'|
|2  |'Casino'    |'Cake'|15.95|'Food'|
|3  |'Opera'     |'Cake'|15.95|'Food'|
|4  |'Strawberry'|'Cake'|11.95|'Food'|
+---+------------+------+-----+------+
only showing top 5 rows



In [42]:
df.show(5, truncate=False)

+-------------+------+--------+
|TransactionID|FoodID|FlavorID|
+-------------+------+--------+
|1            |5     |11      |
|2            |1     |7       |
|2            |3     |11      |
|2            |4     |37      |
|2            |3     |45      |
+-------------+------+--------+
only showing top 5 rows



In [43]:
goods_df.createOrReplaceTempView('goods')

In [44]:
rawData = spark.sql('select g.Flavor, o.TransactionID, o.FlavorID from goods g inner join order_items_train o where o.FlavorID = g.Id')

rawData.show()

+------------+-------------+--------+
|      Flavor|TransactionID|FlavorID|
+------------+-------------+--------+
|     'Apple'|            1|      11|
|    'Coffee'|            2|       7|
|     'Apple'|            2|      11|
|    'Almond'|            2|      37|
|       'Hot'|            2|      45|
|     'Opera'|            3|       3|
|    'Orange'|            3|      42|
|    'Cheese'|            3|      33|
|   'Truffle'|            4|       5|
|     'Apple'|            4|      12|
|   'Vanilla'|            4|      47|
| 'Chocolate'|            4|      17|
|    'Orange'|            5|      42|
| 'Chocolate'|            5|       6|
|    'Cherry'|            5|      18|
|    'Casino'|            6|       2|
|'Strawberry'|            6|       4|
| 'Chocolate'|            6|      34|
|'Blackberry'|            7|      15|
| 'Blueberry'|            7|      16|
+------------+-------------+--------+
only showing top 20 rows



In [45]:
baskets = rawData.groupBy('TransactionID').agg(collect_set('Flavor').alias('items'))
baskets.show()

+-------------+--------------------+
|TransactionID|               items|
+-------------+--------------------+
|          148|['Cheese', 'Tuile...|
|          463|['Chocolate', 'Be...|
|          471|['Almond', 'Pecan...|
|          496|['Vanilla', 'Blac...|
|          833|['Ganache', 'Appl...|
|         1088|['Marzipan', 'Ope...|
|         1238|['Apricot', 'Cher...|
|         1342|['Vanilla', 'Choc...|
|         1580|           ['Apple']|
|         1591|           ['Lemon']|
|         1645|['Blackberry', 'C...|
|         1829|['Almond', 'Black...|
|         1959|['Raspberry', 'Na...|
|         2122|['Truffle', 'Gong...|
|         2142|['Bottled', 'Berr...|
|         2366|['Walnut', 'Choco...|
|         2659|          ['Orange']|
|         2866|['Tuile', 'Casino...|
|         3175|['Cheese', 'Orange']|
|         3749|['Apple', 'Strawb...|
+-------------+--------------------+
only showing top 20 rows



## Áp dụng model

In [46]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.003)
model = fpGrowth.fit(baskets)

In [47]:
# Display frequent itemsets
model.freqItemsets.show(truncate=False)

+---------------------------------+-----+
|items                            |freq |
+---------------------------------+-----+
|['Chocolate']                    |21609|
|['Lemon']                        |16445|
|['Lemon', 'Chocolate']           |4251 |
|['Apple']                        |16109|
|['Apple', 'Lemon']               |2902 |
|['Apple', 'Lemon', 'Chocolate']  |860  |
|['Apple', 'Chocolate']           |3908 |
|['Apricot']                      |15648|
|['Apricot', 'Apple']             |2502 |
|['Apricot', 'Apple', 'Lemon']    |526  |
|['Apricot', 'Apple', 'Chocolate']|720  |
|['Apricot', 'Lemon']             |2562 |
|['Apricot', 'Lemon', 'Chocolate']|709  |
|['Apricot', 'Chocolate']         |3486 |
|['Almond']                       |14278|
|['Almond', 'Apple']              |5002 |
|['Almond', 'Apple', 'Lemon']     |696  |
|['Almond', 'Apple', 'Chocolate'] |916  |
|['Almond', 'Lemon']              |2890 |
|['Almond', 'Lemon', 'Chocolate'] |869  |
+---------------------------------

In [48]:
mostPopularItemInABasket = model.transform(baskets)

In [ ]:
mostPopularItemInABasket1.head(3)

[Row(TransactionID=148, items=["'Cheese'", "'Tuile'", "'Napoleon'", "'Strawberry'", "'Chocolate'", "'Marzipan'"], prediction=["'Apple'", "'Vanilla'", "'Almond'", "'Lemon'", "'Apricot'", "'Coffee'", "'Blueberry'", "'Hot'", "'Raspberry'", "'Cherry'", "'Orange'", "'Gongolais'", "'Berry'", "'Truffle'", "'Opera'", "'Blackberry'", "'Bottled'", "'Casino'", "'Single'", "'Walnut'", "'Green'", "'Pecan'", "'Ganache'"]),
 Row(TransactionID=463, items=["'Chocolate'", "'Berry'"], prediction=["'Coffee'", "'Tuile'", "'Apple'", "'Vanilla'", "'Gongolais'", "'Orange'", "'Almond'", "'Blueberry'", "'Lemon'", "'Hot'", "'Raspberry'", "'Apricot'", "'Strawberry'", "'Marzipan'", "'Cherry'", "'Napoleon'", "'Truffle'", "'Cheese'", "'Opera'", "'Blackberry'", "'Bottled'", "'Casino'", "'Single'", "'Walnut'", "'Green'", "'Pecan'", "'Ganache'"]),
 Row(TransactionID=471, items=["'Almond'", "'Pecan'", "'Napoleon'", "'Chocolate'"], prediction=["'Apple'", "'Lemon'", "'Apricot'", "'Vanilla'", "'Cherry'", "'Blueberry'", "'C

In [50]:
mostPopularItemInABasket.createOrReplaceTempView("popular_items")

In [51]:
df_cast = mostPopularItemInABasket.select('TransactionID', mostPopularItemInABasket.items.cast(StringType()),\
                                           mostPopularItemInABasket.prediction.cast(StringType()))

In [52]:
df_cast.printSchema()

root
 |-- TransactionID: integer (nullable = true)
 |-- items: string (nullable = true)
 |-- prediction: string (nullable = true)



In [53]:
df_cast.show()

+-------------+--------------------+--------------------+
|TransactionID|               items|          prediction|
+-------------+--------------------+--------------------+
|          148|['Cheese', 'Tuile...|['Apple', 'Vanill...|
|          463|['Chocolate', 'Be...|['Coffee', 'Tuile...|
|          471|['Almond', 'Pecan...|['Apple', 'Lemon'...|
|          496|['Vanilla', 'Blac...|['Napoleon', 'Cof...|
|          833|['Ganache', 'Appl...|['Lemon', 'Chocol...|
|         1088|['Marzipan', 'Ope...|['Napoleon', 'Cof...|
|         1238|['Apricot', 'Cher...|['Vanilla', 'Almo...|
|         1342|['Vanilla', 'Choc...|['Casino', 'Napol...|
|         1580|           ['Apple']|['Lemon', 'Chocol...|
|         1591|           ['Lemon']|['Chocolate', 'Ap...|
|         1645|['Blackberry', 'C...|['Casino', 'Napol...|
|         1829|['Almond', 'Black...|['Vanilla', 'Appl...|
|         1959|['Raspberry', 'Na...|['Apricot', 'Coff...|
|         2122|['Truffle', 'Gong...|['Coffee', 'Tuile...|
|         2142